In [4]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import pandas as pd
import torchvision.transforms as transforms

class CheXpertDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):

        self.chexpert_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.chexpert_frame)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.chexpert_frame.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        labels = self.chexpert_frame.iloc[idx, 5:].values.astype('float')
        labels[labels == -1] = 1  
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(labels)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

root_dir = '/kaggle/input/chexpert-v10-small/'

train_dataset = CheXpertDataset(
    csv_file='/kaggle/input/chexpert-v10-small/CheXpert-v1.0-small/train.csv',
    root_dir=root_dir,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

images, labels = next(iter(train_loader))
print(images.shape, labels.shape)


torch.Size([16, 3, 224, 224]) torch.Size([16, 14])


In [5]:
import torch
import torchvision.models as models
import torch.nn as nn

local_weights_path = '/kaggle/input/resnet/pytorch/1/1/resnet50-11ad3fa6.pth'

model = models.resnet50(pretrained=False)

model.load_state_dict(torch.load(local_weights_path))

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 14)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(model)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader


optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.BCEWithLogitsLoss()

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()  
    for epoch in range(num_epochs):
        running_loss = 0.0
        total_batches = len(train_loader)
        print(f"Starting Epoch {epoch+1}/{num_epochs}, Total batches: {total_batches}")

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            if (i + 1) % 10 == 0:
                print(f'Epoch {epoch+1}, Batch {i+1}/{total_batches}, Batch Loss: {loss.item():.4f}')

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Total Loss: {epoch_loss:.4f}')


train_model(model, train_loader, criterion, optimizer, num_epochs=10)
